In [1]:
import csv
import numpy as np
import math
import sys

#read data

data = []
# 每一個維度儲存一種污染物的資訊
for i in range(18):
	data.append([])

n_row = 0
text = open('train.csv', 'r', encoding='big5') 
row = csv.reader(text , delimiter=",")
for r in row:
    # 第0列沒有資訊
    if n_row != 0:
        # 每一列只有第3-27格有值(1天內24小時的數值)
        for i in range(3,27):
            if r[i] != "NR":
                data[(n_row-1)%18].append(float(r[i]))
            else:
                data[(n_row-1)%18].append(float(0))	
    n_row = n_row+1

text.close()

#parse data to (x,y)

x = []
y = []
# 每 12 個月
for i in range(12):
    # 一個月取連續10小時的data可以有471筆
    for j in range(471):
        x.append([])
        # 18種污染物
        for t in range(18):
            # 連續9小時
            for s in range(9):
                x[471*i+j].append(data[t][480*i+j+s] )
        y.append(data[9][480*i+j+9])
x = np.array(x)
y = np.array(y)

# add square term
# x = np.concatenate((x,x**2), axis=1)

# add bias
x = np.concatenate((np.ones((x.shape[0],1)),x), axis=1)

#init weight & other hyperparams
w = np.zeros(len(x[0]))
l_rate = 10
repeat = 10000

#check your ans with close form solution
# use close form to check whether ur gradient descent is good
# however, this cannot be used in hw1.sh 
# w = np.matmul(np.matmul(inv(np.matmul(x.transpose(),x)),x.transpose()),y)

#start training 
x_t = x.transpose()
s_gra = np.zeros(len(x[0]))
print(s_gra)
print(x_t)
for i in range(repeat):
    hypo = np.dot(x,w) #dot	函數在一維向量為內稽，二維時為正常矩陣相乘(與*相同);multiply則為entry相乘
    loss = hypo - y
    cost = np.sum(loss**2) / len(x)
    cost_a  = math.sqrt(cost)
    gra = np.dot(x_t,loss)
    s_gra += gra**2
    ada = np.sqrt(s_gra)
    w = w - l_rate * gra/ada
    #print ('iteration: %d | Cost: %f  ' % ( i,cost_a))

# save model
np.save('model.npy',w)
# read model
w = np.load('model.npy')

#read testing data
test_x = []
n_row = 0
text = open('test.csv' ,"r")
row = csv.reader(text , delimiter= ",")
print(w)
for r in row:
    if n_row %18 == 0:
        test_x.append([])
        for i in range(2,11):
            test_x[n_row//18].append(float(r[i]) )
    else :
        for i in range(2,11):
            if r[i] !="NR":
                test_x[n_row//18].append(float(r[i]))
            else:
                test_x[n_row//18].append(0)
    n_row = n_row+1
text.close()
test_x = np.array(test_x)

# add square term
# test_x = np.concatenate((test_x,test_x**2), axis=1)

# add bias
test_x = np.concatenate((np.ones((test_x.shape[0],1)),test_x), axis=1)

#get ans.csv with your model
ans = []
for i in range(len(test_x)):
    ans.append(["id_"+str(i)])
    a = np.dot(w,test_x[i])
    ans[i].append(a)

filename = "predict.csv"
text = open(filename, "w+")
s = csv.writer(text,delimiter=',',lineterminator='\n')
s.writerow(["id","value"])
for i in range(len(ans)):
    s.writerow(ans[i]) 
text.close()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[ 1.   1.   1.  ...  1.   1.   1. ]
 [14.  13.  13.  ... 21.  21.  22. ]
 [13.  13.  12.  ... 21.  22.  21. ]
 ...
 [ 0.4  0.5  0.8 ...  1.   0.8  0.4]
 [ 0.5  0.8  0.8 ...  0.8  0.4  1.1]
 [ 0.8  0.8  0.5 ...  0.4  1.1  1.1]]
[ 2.54076977e-02 -2.20947067e-02 -5.50897814e-03  4.04126682e-02
  9.05866323e-03  1.74051999e-02 -9.41232449e-04 -2.68493029e-02
 -3.54069918e-02 -2.10050620e-02  1.21122768e-02 -2.73389556e-02
 -2.60922920e-02  2.29273428e-02  6.53181125e-03 -2.10707907e-03
  2.91086251e-0

In [4]:

import sys
import math
import pandas as pd
import numpy as np

attrs = ['AMB', 'CH4', 'CO', 'NMHC', 'NO', 'NO2',
        'NOx', 'O3', 'PM10', 'PM2.5', 'RAINFALL', 'RH',
        'SO2', 'THC', 'WD_HR', 'WIND_DIR', 'WIND_SPEED', 'WS_HR']

attr_range = {}

for i, attr in enumerate(attrs):
    attr_range[attr] = list(range(9*i, 9*i+9))

def ReadTrainData(filename):
    raw_data = pd.read_csv(filename, encoding='big5').as_matrix()
    data = raw_data[:, 3:] # 12 months, 20 days per month, 18 features per day. shape: (4320 , 24)
    data[data == 'NR'] = 0.0
    data = data.astype('float')

    X, Y = [], []
    for i in range(0, data.shape[0], 18*20):
        # i: start of each month
        days = np.vsplit(data[i:i+18*20], 20) # shape: 20 * (18, 24)
        concat = np.concatenate(days, axis=1) # shape: (18, 480)
        for j in range(0, concat.shape[1]-9):
            X.append(concat[:, j:j+9].flatten())
            Y.append([concat[9, j+9]])

    return np.array(X), np.array(Y)

def ReadTestData(filename):
    raw_data = pd.read_csv(filename, header=None, encoding='big5').as_matrix()
    data = raw_data[:, 2:]
    data[data == 'NR'] = 0.0
    data = data.astype('float')

    obs = np.vsplit(data, data.shape[0]/18)
    X = []
    for i in obs:
        X.append(i.flatten())

    return np.array(X)

class Linear_Regression():
    def __init__(self):
        pass

    def _error(self, X, Y):
        return Y - self.predict(X)

    def _loss(self, X, Y):
        return np.sqrt(np.mean(self._error(X, Y) ** 2))

    def _scale(self, X, istrain=True):
        if istrain:
            self.mean = np.mean(X, axis=0)
            self.std = np.std(X, axis=0) + 1e-20
        return (X - self.mean) / self.std

    def _add_bias(self, X):
        return np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    def fit(self, _X, Y, valid):
        assert _X.shape[0] == Y.shape[0]
        N = _X.shape[0]
        self.feature_dim = feature_dim = _X.shape[1]

        X = self._scale(_X)
        X_bias = self._add_bias(X)

        #self.W = np.linalg.lstsq(X_bias, Y)[0]
        self.W = np.dot(np.dot(np.linalg.inv(np.dot(X_bias.T, X_bias)), X_bias.T), Y)
        print('training loss: {}'.format(self._loss(X, Y)))

        if valid is not None:
            X_valid, Y_valid = valid
            X_valid = self._scale(X_valid, istrain=False)
            print('valid loss: {}'.format(self._loss(X_valid, Y_valid)))
            

    def predict(self, X):
        _X = np.reshape(X, (-1, self.feature_dim))
        _X = self._add_bias(_X)
        return np.dot(_X, self.W)
 
    def predict_test(self, X):
        _X = self._scale(X, istrain=False)
        _X = _X.reshape((-1, self.feature_dim))
        _X = self._add_bias(_X)
        return np.dot(_X, self.W)

def main(args):
    X, Y = ReadTrainData(args[1])
    X_test = ReadTestData(args[2])

    select_attr = attrs
    select_attr = ['PM10', 'PM2.5', 'O3', 'WIND_DIR', 'WIND_SPEED', 'WD_HR', 'WS_HR', 'RAINFALL', 'CO']
    select_range = []
    for attr in select_attr:
        select_range += attr_range[attr]

    X = X[:, select_range]
    X_test = X_test[:, select_range]

    X = np.concatenate((X, X[:, 0:18] ** 3, np.multiply(X[:, 9:18], X[:, 18:27])), axis=1)
    X_test = np.concatenate((X_test, X_test[:, 0:18] ** 3, np.multiply(X_test[:, 9:18], X_test[:, 18:27])), axis=1)

    valid = None
    try:
        valid_num = int(args[4])
        order = np.random.permutation(X.shape[0])
    except:
        coef = np.loadtxt(args[4])
        valid_num = coef[0].astype('int')
        order = coef[1:].astype('int')
    X, Y = X[order], Y[order]
    valid = X[:valid_num], Y[:valid_num]
    X, Y = X[valid_num:], Y[valid_num:]

    model = Linear_Regression()
    model.fit(X, Y, valid=valid)

    predict = model.predict_test(X_test)
    with open(args[3], 'w') as f:
        print('id,value', file=f)
        for (i, p) in enumerate(predict) :
            print('id_{},{}'.format(i, p[0]), file=f)

    np.savetxt('coef.txt', np.concatenate((np.asarray(valid_num).reshape((1, )), order)), fmt='%d')

if __name__ == '__main__':
    main(sys.argv)


FileNotFoundError: File b'-f' does not exist